# spacy + cuDNNLSTM Implementation for Sentiment Analysis
The [data](https://www.kaggle.com/bittlingmayer/amazonreviews) is downloaded from Kaggle, and data cleaning and preprocessing uses the codes modified from the [kernel by Anshul Rai](https://www.kaggle.com/anshulrai/cudnnlstm-implementation-93-7-accuracy).

In [5]:
import SentimentAnalyser # spaCy
import numpy as np 
import pandas as pd 
import bz2
import gc # garbage collector, release unused memory
import chardet
import re
import os
print(os.listdir("./input"))
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Conv1D, GlobalMaxPool1D, Dropout, concatenate, Layer, InputSpec, CuDNNLSTM
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import activations, initializers, regularizers, constraints
from keras.utils.conv_utils import conv_output_length
from keras.regularizers import l2
from keras.constraints import maxnorm



['train.ft.txt', 'test.ft.txt.bz2', 'test.ft.txt', 'train.ft.txt.bz2', 'amazonreviews.zip']


In [24]:
test_file = bz2.BZ2File('./input/test.ft.txt.bz2')
train_file = bz2.BZ2File('./input/train.ft.txt.bz2')

In [26]:
def readbinary2list(file): 
    lines = file.readlines()
    lines = [line.decode('utf-8') for line in lines]
    labels = [0 if line.split(' ')[0] == '__label__1' else 1 for line in lines]
    sentences = [line.split(' ', 1)[1][:-1].lower() for line in lines]
    for i in range(len(sentences)):
        sentences[i] = re.sub('\d','0',sentences[i])
        if 'www.' in sentences[i] or 'http:' in sentences[i] or 'https:' in sentences[i] or '.com' in sentences[i]:
            sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", sentences[i])    
    return labels, sentences

In [27]:
test_labels,test_sentences = readbinary2list(test_file)
train_labels,train_sentences = readbinary2list(train_file)

In [48]:
i=0
print('label: %s \n\nsentences: \n%s\n'%(train_labels[i],train_sentences[i]))

i=6
print('label: %s \n\nsentences: \n%s\n'%(train_labels[i],train_sentences[i]))

label: 1 

sentences: 
stuning even for the non-gamer: this sound track was beautiful! it paints the senery in your mind so well i would recomend it even to people who hate vid. game music! i have played the game chrono cross but out of all of the games i have ever played it has the best music! it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. it would impress anyone who cares to listen! ^_^

label: 0 

sentences: 
buyer beware: this is a self-published book, and if you want to know why--read a few paragraphs! those 0 star reviews must have been written by ms. haddon's family and friends--or perhaps, by herself! i can't imagine anyone reading the whole thing--i spent an evening with the book and a friend and we were in hysterics reading bits and pieces of it to one another. it is most definitely bad enough to be entered into some kind of a "worst book" contest. i can't believe amazon even sells this kind of thing. maybe i can offer

In [47]:
gc.collect() # free memory

1089

In [ ]:
max_features = 20000
maxlen = 100

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)

# Reference
* [cuDNNLSTM keras doc](https://keras.io/layers/recurrent/)
* [cuDNNLSTM implement project](https://www.kaggle.com/anshulrai/cudnnlstm-implementation-93-7-accuracy)
* [spaCy sentiment git](https://github.com/explosion/spaCy/blob/master/examples/deep_learning_keras.py)
* [spaCy](https://spacy.io/models/)
* [Python kaggle docker](https://github.com/Kaggle/docker-python/blob/master/Dockerfile)